In [1]:
data_path = "./data/Rotten Tomatoes Movies.csv"
image_path = "./base de conhecimento 2/Valles Marineris.jpeg"
notebook_path = "./base de conhecimento 2/notebooks/analise_meteorologica.ipynb"

In [2]:
from file_interaction import DataReaderAgent, NotebookReaderAgent, ImageReaderAgent

In [3]:
# utilizando o gpt
from langchain_openai import ChatOpenAI

gpt = ChatOpenAI(model="gpt-4o-mini")

In [4]:
# utilizando o llama3.1 8b
from langchain_ollama import ChatOllama

llama3 = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
    format='json'
)

## Data files

In [5]:
from utils import State
# Um estado inicial para preencher o contexto
state = {
    "main_prompt": "The objective is to provide the necessary result to answer the specific query",
    "dynamic_context": ""
}
state = State(**state)

data_agent_gpt = DataReaderAgent(gpt, "./prompts/context_from_dataframe.jinja2")
data_agent_llama3 = DataReaderAgent(llama3, "./prompts/context_from_dataframe.jinja2")

In [7]:
prompts = [
    "What is the most well tomatometer rated movie from the most recent year in the data?",
    "What is the oldest year in the data?",
    "What is the movie with the highest tomatometer rating in the data?",
    "What is the movie with the lowest tomatometer rating in the data?",
    "What director has the most movies in the data?",
]

for prompt in prompts:
    answer = data_agent_gpt.get_context_from_file(prompt, data_path, state)
    result = answer.split("and the result of the query is:")[1].strip()
    sql = answer.split("```sql")[1].split("```")[0].strip()
    print(f"Prompt: {prompt}")
    print("Using GPT-4o-mini")
    print(f"SQL: {sql}")
    print(f"Result: {result}")
    print("\n")
    answer = data_agent_llama3.get_context_from_file(prompt, data_path, state)
    result = answer.split("and the result of the query is:")[1].strip()
    sql = answer.split("```sql")[1].split("```")[0].strip()
    print("Using llama3.1 8b")
    print(f"Prompt: {prompt}")
    print(f"SQL: {sql}")
    print(f"Result: {result}")
    print("\n")

Prompt: What is the most well tomatometer rated movie from the most recent year in the data?
Using GPT-4o-mini
SQL: SELECT movie_title, tomatometer_rating FROM rotten_tomatoes_movies_table WHERE strftime('%Y', in_theaters_date) = (SELECT MAX(strftime('%Y', in_theaters_date)) FROM rotten_tomatoes_movies_table) ORDER BY tomatometer_rating DESC LIMIT 1;
Result: movie_title  tomatometer_rating
0          5B                 100


Using llama3.1 8b
Prompt: What is the most well tomatometer rated movie from the most recent year in the data?
SQL: SELECT movie_title, tomatometer_rating FROM rotten_tomatoes_movies_table WHERE YEAR(in_theaters_date) = ( SELECT MAX(YEAR(in_theaters_date)) FROM rotten_tomatoes_movies_table ) ORDER BY tomatometer_rating DESC LIMIT 1;
Result: An error occurred while executing the query: (sqlite3.OperationalError) no such function: YEAR
[SQL: SELECT movie_title, tomatometer_rating FROM local_df_variable WHERE YEAR(in_theaters_date) = ( SELECT MAX(YEAR(in_theaters_date

## Images

In [5]:
from utils import State
# Um estado inicial para preencher o contexto
state = {
    "main_prompt": "The objective is to understant everything we can about the landing",
    "dynamic_context": ""
}
state = State(**state)

image_agent_gpt = ImageReaderAgent(gpt, '')

relevant_content = image_agent_gpt.get_context_from_file("I want every detail related to the landing from this image", image_path, state)
print(relevant_content)


        [[Visual Analysis of the image at ./base de conhecimento 2/Valles Marineris.jpeg]]
        
            Prompt: 'I want every detail related to the landing from this image'
            
            Result:
            To analyze the image related to the Noctis Landing on Mars, the following details can be extracted:

1. **Location**: 
   - The image depicts the Noctis Landing site, indicated by a black dot within an oval. This is situated in the Noctis Labyrinthus region, which is characterized by its complex series of valleys and canyons.

2. **Surrounding Geological Features**:
   - **Tharsis**: A large volcanic region to the north, known for its massive volcanoes.
   - **Valles Marineris**: A prominent canyon system, one of the largest in the solar system, located to the right of the landing site.
   - **Chasmas**: 
     - **Tithonium Chasma** and **Ius Chasma** are specific valleys that are part of the Valles Marineris system, which may provide insights into geological pro

## Notebooks

Buscando no notebook, um entendimento sobre a relação de temperatura e geolocalização

In [5]:
from utils import State
# Um estado inicial para preencher o contexto
state = {
    "main_prompt": "The objective is to understant how the temperature correlates with the coordinates",
    "dynamic_context": ""
}
state = State(**state)

notebook_agent_gpt = NotebookReaderAgent(gpt, "./prompts/context_from_notebook_file.jinja2")

In [6]:
relevant_content = notebook_agent_gpt.get_context_from_file(specific_prompt="I want to know whats the relationship between temperature and coordinates", file_path=notebook_path, state=state)

In [9]:
print(relevant_content)

The analysis in the notebook includes loading meteorological data that contains temperatures recorded at different locations. It emphasizes understanding temperature variations by location, as multiple sensors provide data from various places on different dates. The notebook includes visualizations such as:

1. A plot of temperature over time, showing how temperature varies through the recorded dates.
2. A scatter plot that displays temperature variations based on coordinates (longitude and latitude), where the color of the points represents temperature. This visualization directly relates to the relationship between temperature and coordinates.
3. A heatmap generated from the temperature data across coordinates, providing a more continuous representation of temperature distribution over the geographical area.

These visualizations help illustrate how temperature correlates with specific coordinates, indicating spatial temperature patterns.

[description of image with figure_id = figur